# Glands Segmentator #


In this file, I use two models that I created to successfully perform gland segmentation. The process forms a pipeline where an input image is provided for segmentation, and the output is an image with the segmented glands.

# Procesing of big images #

Imports:

In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import tifffile

Loading models:

In [6]:
import unet_core

print(unet_core.__file__)  # Pokazuje lokalizację faktycznego pliku
dir(unet_core)



C:\Users\stszy\miniconda3\Lib\site-packages\unet_core\__init__.py


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [7]:
import unet_core.unet_interface

print(dir(unet_core.unet_interface))


['Image', 'UNET', 'UNetArchitecture', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'cv2', 'get_binary_mask', 'get_contours_from_mask', 'np', 'prepare_image', 'torch', 'train_process']


C:\Users\stszy\AppData\Roaming\Python\Python312\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [8]:
import importlib.util

spec = importlib.util.find_spec("unet_core.unet_interface")
print(spec)


ModuleSpec(name='unet_core.unet_interface', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000023432B10860>, origin='C:\\Users\\stszy\\miniconda3\\Lib\\site-packages\\unet_core\\unet_interface.py')


In [9]:
detectionModel = YOLO("../model/saved_models/Glands_Finder_Augumented_Data_best.pt")
from unet_core.unet_interface import UNET

modelUNET = UNET(img_height=50, img_width=50)
modelUNET.set_model(in_channels=3, out_channels=1,
                    name="C:/Users/stszy/miniconda3/Lib/site-packages/unet_core/GlandsFinder")


Chosen model successfully found
=> Loading checkpoint


 Helper function:

In [38]:
def tile_image(image, tile_size, overlap):
    tiles = []
    positions = []
    h, w = image.shape[:2]
    stride = tile_size - overlap

    for y in range(0, h, stride):
        for x in range(0, w, stride):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            positions.append((x, y))
    return tiles, positions


def detect_on_tiles(tiles, positions, model):
    all_detections = []
    for tile, (x_offset, y_offset) in zip(tiles, positions):
        results = model(tile, conf=0.6)[0]
        for box in results.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            x1 += x_offset
            x2 += x_offset
            y1 += y_offset
            y2 += y_offset
            conf = float(box.conf)
            cls = int(box.cls)
            all_detections.append((x1, y1, x2, y2, conf, cls))
    return all_detections


def draw_detections(image, detections, class_names):
    annotated = image.copy()
    for x1, y1, x2, y2, conf, cls in detections:
        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        label = f"{class_names[cls]} {conf:.2f}"
        cv2.putText(annotated, label, (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return annotated


import math


def merge_overlapping_boxes(
        detections,
        merge_distance=80,
        size_similarity_thresh=0.3
):
    merged = []
    used = [False] * len(detections)

    for i, det in enumerate(detections):
        if used[i]:
            continue

        x1, y1, x2, y2, conf, cls = det
        w1 = x2 - x1
        h1 = y2 - y1
        cx1 = (x1 + x2) / 2
        cy1 = (y1 + y2) / 2
        group = [(x1, y1, x2, y2, conf)]
        used[i] = True

        for j in range(i + 1, len(detections)):
            if used[j]:
                continue

            x1_b, y1_b, x2_b, y2_b, conf_b, cls_b = detections[j]
            w2 = x2_b - x1_b
            h2 = y2_b - y1_b
            cx2 = (x1_b + x2_b) / 2
            cy2 = (y1_b + y2_b) / 2

            # 1. Środki blisko siebie
            dist = math.hypot(cx1 - cx2, cy1 - cy2)
            if dist > merge_distance:
                continue

            # 2. Jeden z wymiarów podobny
            width_similar = abs(w1 - w2) / max(w1, w2) <= size_similarity_thresh
            height_similar = abs(h1 - h2) / max(h1, h2) <= size_similarity_thresh
            if not (width_similar or height_similar):
                continue

            # Pasuje — dodaj do grupy
            group.append((x1_b, y1_b, x2_b, y2_b, conf_b))
            used[j] = True

        # Scal grupę
        xs1, ys1, xs2, ys2, confs = zip(*group)
        merged_box = (
            min(xs1),
            min(ys1),
            max(xs2),
            max(ys2),
            sum(confs) / len(confs),
            cls
        )
        merged.append(merged_box)

    return merged


import random


def draw_detections_varied_colors(image, detections, class_names, thickness=2, font_scale=0.5):
    annotated = image.copy()
    random.seed(42)

    for idx, det in enumerate(detections):
        x1, y1, x2, y2, conf, cls_id = det

        color = tuple(int(c) for c in np.random.randint(0, 256, size=3))

        label = f"{class_names[int(cls_id)]} {conf:.2f}"

        cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
        text_y = int(y1) - 5 if y1 > 10 else int(y1) + 15
        cv2.putText(
            annotated,
            label,
            (int(x1), text_y),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            color,
            thickness=1,
            lineType=cv2.LINE_AA
        )

    return annotated


In [39]:


image_path = "../preprocessedData/tissue_regions/1M01/tissue_region_0.tiff"
image = tifffile.imread(image_path)
if image.ndim == 3 and image.shape[2] > 3:
    image = image[:, :, :3]

if image.ndim == 2:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

tile_size = 2048
overlap = 1024
tiles, positions = tile_image(image, tile_size, overlap)

detections = detect_on_tiles(tiles, positions, detectionModel)

detections = merge_overlapping_boxes(detections, merge_distance=80)

class_names = detectionModel.names

annotated = draw_detections(image, detections, class_names)

cv2.imwrite("wynik_z_detections.png", annotated)

annotated = draw_detections_varied_colors(image, detections, class_names)
cv2.imwrite("wynik_z_detections_varied_colors.png", annotated)



0: 640x640 (no detections), 85.3ms
Speed: 3.5ms preprocess, 85.3ms inference, 33.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 87.9ms
Speed: 3.4ms preprocess, 87.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 75.5ms
Speed: 3.3ms preprocess, 75.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 79.4ms
Speed: 3.3ms preprocess, 79.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 79.3ms
Speed: 3.9ms preprocess, 79.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 82.1ms
Speed: 3.3ms preprocess, 82.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 82.4ms
Speed: 3.1ms preprocess, 82.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 gland, 80.2ms
Speed: 4.4ms preprocess, 80.2ms inferenc

True

In [12]:
import numpy as np


def tile_image(image, tile_size, overlap):
    tiles = []
    h, w = image.shape[:2]
    stride = tile_size - overlap

    ys = list(range(0, h, stride))
    xs = list(range(0, w, stride))

    n_tiles_y = len(ys)
    n_tiles_x = len(xs)

    positions_matrix = np.empty((n_tiles_y, n_tiles_x), dtype=object)

    for i, y in enumerate(ys):
        for j, x in enumerate(xs):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            positions_matrix[i, j] = (x, y)

    return tiles, positions_matrix


In [16]:
image_path = "../preprocessedData/tissue_regions/1M01/tissue_region_0.tiff"
image = tifffile.imread(image_path)
if image.ndim == 3 and image.shape[2] > 3:
    image = image[:, :, :3]

if image.ndim == 2:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

tile_size = 2048
overlap = 1024

class_names = detectionModel.names
tiles, positions_matrix = tile_image(image, tile_size, overlap)

In [17]:
print("Number of tiles:", len(tiles))
print("Positions matrix shape:", positions_matrix.shape)
print(positions_matrix)

Number of tiles: 304
Positions matrix shape: (16, 19)
[[(0, 0) (1024, 0) (2048, 0) (3072, 0) (4096, 0) (5120, 0) (6144, 0) (7168, 0) (8192, 0) (9216, 0) (10240, 0) (11264, 0) (12288, 0) (13312, 0) (14336, 0) (15360, 0) (16384, 0) (17408, 0) (18432, 0)]
 [(0, 1024) (1024, 1024) (2048, 1024) (3072, 1024) (4096, 1024) (5120, 1024) (6144, 1024) (7168, 1024) (8192, 1024) (9216, 1024) (10240, 1024) (11264, 1024) (12288, 1024) (13312, 1024) (14336, 1024) (15360, 1024) (16384, 1024) (17408, 1024) (18432, 1024)]
 [(0, 2048) (1024, 2048) (2048, 2048) (3072, 2048) (4096, 2048) (5120, 2048) (6144, 2048) (7168, 2048) (8192, 2048) (9216, 2048) (10240, 2048) (11264, 2048) (12288, 2048) (13312, 2048) (14336, 2048) (15360, 2048) (16384, 2048) (17408, 2048) (18432, 2048)]
 [(0, 3072) (1024, 3072) (2048, 3072) (3072, 3072) (4096, 3072) (5120, 3072) (6144, 3072) (7168, 3072) (8192, 3072) (9216, 3072) (10240, 3072) (11264, 3072) (12288, 3072) (13312, 3072) (14336, 3072) (15360, 3072) (16384, 3072) (17408, 

In [31]:
def tile_image_with_detection(image, tile_size, overlap, model):
    tiles = []
    h, w = image.shape[:2]
    stride = tile_size - overlap

    ys = list(range(0, h, stride))
    xs = list(range(0, w, stride))

    n_tiles_y = len(ys)
    n_tiles_x = len(xs)

    detection_matrix = np.empty((n_tiles_y, n_tiles_x), dtype=object)

    for i, y in enumerate(ys):
        for j, x in enumerate(xs):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            results = model(tile, conf=0.6)[0]
            list_of_boxes = []
            for box in results.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                x1 += x
                x2 += x
                y1 += y
                y2 += y
                conf = float(box.conf)
                cls = int(box.cls)
                list_of_boxes.append((x1, y1, x2, y2, conf, cls))

            detection_matrix[i, j] = (x, y, list_of_boxes)

    return tiles, detection_matrix


In [32]:
image_path = "../preprocessedData/tissue_regions/1M01/tissue_region_0.tiff"
image = tifffile.imread(image_path)
if image.ndim == 3 and image.shape[2] > 3:
    image = image[:, :, :3]

if image.ndim == 2:
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

tile_size = 2048
overlap = 1024

class_names = detectionModel.names
tiles, positions_matrix = tile_image_with_detection(image, tile_size, overlap, detectionModel)


0: 640x640 (no detections), 88.0ms
Speed: 3.5ms preprocess, 88.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 88.0ms
Speed: 3.8ms preprocess, 88.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 80.8ms
Speed: 3.5ms preprocess, 80.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 78.4ms
Speed: 3.5ms preprocess, 78.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 77.2ms
Speed: 3.6ms preprocess, 77.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 76.6ms
Speed: 3.7ms preprocess, 76.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 84.5ms
Speed: 3.4ms preprocess, 84.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 gland, 79.1ms
Speed: 3.8ms preprocess, 79.1ms inference

In [22]:
print("Number of tiles:", len(tiles))
print("Positions matrix shape:", positions_matrix.shape)
print(positions_matrix)

Number of tiles: 304
Positions matrix shape: (16, 19)
[[None None None None None None None (7168, 0, [(np.float32(8984.985), np.float32(1596.1863), np.float32(9215.905), np.float32(2048.0), 0.7388763427734375, 0)])
  (8192, 0, [(np.float32(9686.097), np.float32(1174.5686), np.float32(10237.562), np.float32(1626.5875), 0.8876985311508179, 0), (np.float32(9958.082), np.float32(1882.0026), np.float32(10239.806), np.float32(2047.242), 0.6960609555244446, 0), (np.float32(9961.635), np.float32(1547.8596), np.float32(10239.856), np.float32(2041.3541), 0.6890297532081604, 0)])
  (9216, 0, [(np.float32(9958.877), np.float32(1552.4283), np.float32(10544.705), np.float32(2036.5139), 0.9550598859786987, 0), (np.float32(9675.885), np.float32(1169.074), np.float32(10241.976), np.float32(1621.9808), 0.9130753874778748, 0), (np.float32(10323.123), np.float32(1476.7266), np.float32(10739.624), np.float32(1878.3828), 0.9123066067695618, 0)])
  (10240, 0, [(np.float32(10302.937), np.float32(1469.0115), n

In [40]:
def draw_detections_from_matrix(image, detection_matrix, class_names):
    annotated = image.copy()
    for i in range(detection_matrix.shape[0]):
        for j in range(detection_matrix.shape[1]):
            x, y, detections = detection_matrix[i, j]

            if detections is not None:
                for det in detections:
                    x1, y1, x2, y2, conf, cls = det
                    cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    label = f"{class_names[cls]} {conf:.2f}"
                    cv2.putText(annotated, label, (int(x1), int(y1) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    return annotated



In [41]:
annotated = draw_detections_from_matrix(image, positions_matrix, class_names)

cv2.imwrite("matrix.png", annotated)


True